In [1]:
import pandas as pd
import numpy as np

In [2]:
training_data = pd.read_csv('cleaned_training_set.csv')

In [3]:
training_data.shape

(718637, 24)

In [4]:
training_data.drop("Unnamed: 0", axis=1, inplace=True)

In [5]:
training_data.head()

,Store,DayOfWeek,day,month,year,Sales,Customers,Open,Promo,SchoolHoliday,...,storeType_a,storeType_b,storeType_c,storeType_d,Assortment_a,Assortment_b,Assortment_c,public_holiday,easter,christmas
0,1,5,31,7,2015,5263,555,1,1,1,...,0,0,1,0,1,0,0,0,0,0
1,2,5,31,7,2015,6064,625,1,1,1,...,1,0,0,0,1,0,0,0,0,0
2,3,5,31,7,2015,8314,821,1,1,1,...,1,0,0,0,1,0,0,0,0,0
3,4,5,31,7,2015,13995,1498,1,1,1,...,0,0,1,0,0,0,1,0,0,0
4,5,5,31,7,2015,4822,559,1,1,1,...,1,0,0,0,1,0,0,0,0,0


In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [7]:
Y = training_data.Sales.to_frame()
X = training_data
X.drop('Sales',axis=1,inplace=True)
X = X.to_numpy()
Y = Y.to_numpy()

In [8]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3) 
TimeSeriesSplit(max_train_size=None, n_splits=3)
counter=0
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    name='X_train_'+str(counter)  
    np.save(name, X_train)
    name='X_test_'+str(counter) 
    np.save(name, X_test)
    name='y_test_'+str(counter) 
    np.save(name, y_test)
    name='y_train'+str(counter)  
    np.save(name, y_train)
    counter+=1

TRAIN: [     0      1      2 ... 179657 179658 179659] TEST: [179660 179661 179662 ... 359316 359317 359318]
TRAIN: [     0      1      2 ... 359316 359317 359318] TEST: [359319 359320 359321 ... 538975 538976 538977]
TRAIN: [     0      1      2 ... 538975 538976 538977] TEST: [538978 538979 538980 ... 718634 718635 718636]


In [10]:
est = GradientBoostingRegressor(n_estimators=400, learning_rate=0.1, \
                                max_depth=2, random_state=0, loss='ls').fit(X_train, y_train)

print('This is the MSE: ' + str(mean_squared_error(y_test, est.predict(X_test))))

/Users/mackdelany/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


This is the MSE: 979663.8332600966


In [18]:
est

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=1, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [9]:
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

In [11]:
def run_GBT(n_estimators, learning_rate, max_depth, random_state, loss):
    
    test_err = []
    train_err = []
    
    ytrain = 'y_train{}.npy'
    xtrain = 'X_train_{}.npy'
    ytest = 'y_test_{}.npy'
    xtest = 'X_test_{}.npy'
    
    for i in range(3): 

        y_train = np.load(ytrain.format(i)).reshape(-1)
        name = 'X_train_'+ str(i) + '.npy'
        x_train = np.load(xtrain.format(i)).reshape(len(y_train),-1)
        
        model = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, \
                                max_depth=max_depth, random_state=random_state, loss=loss)
              
        model.fit(x_train, y_train)
        y_test = np.load(ytest.format(i)).reshape(-1)
        x_test = np.load(xtest.format(i)).reshape(len(y_test),-1)
        y_hat = model.predict(x_test)
        
        print(y_hat)
        idx = y_test != 0
        test_err.append(metric (y_test[idx],y_hat[idx]))
        y_hat = model.predict(x_train)
        idx = y_train != 0
        train_err.append(metric (y_train[idx],y_hat[idx]))
        
   
    return test_err,train_err

In [18]:
run_GBT(n_estimators=400, learning_rate=0.1, max_depth=2, random_state=0, loss='ls')

[ 3749.26864256 10752.51008121  4319.47505261 ...  9881.18361201
  4044.93337388  3192.14110707]
[9023.07659477 8922.2953839  4812.74750037 ... 4148.57274423 6166.64379608
 5315.02844941]
[6396.25942203 7092.10561544 6795.68562515 ... 5938.65208787 8278.33826191
 7109.3711632 ]


([14.029952606639222, 13.753533990527195, 14.859388329825666],
 [13.43220916767162, 13.698780274906122, 13.80652934415835])

In [4]:
results = pd.DataFrame(columns = ['n_estimators','learning_rate','max_depth',''])

results

,n_estimators,learning_rate,max_depth


In [ ]:
for n_estimators in [10, 300, 400]:
    for learning_rate in [0.75, 0.2, 0.3]:
        for max_depth in [2, 3, 4]:
            print('Running GBT for...')
            print('n_estimators: ' + str(n_estimators))
            print('learning_rate:' + str(learning_rate))
            print('max_depth:' + str (max_depth))
            test_err,train_err = run_GBT(n_estimators, learning_rate, max_depth, random_state=0, loss='ls')
            print('Test Error:')
            print(test_err)
            print('Training Error')
            print(train_err)


Running GBT for...
n_estimators: 10
learning_rate:0.75
max_depth:2
[ 4098.33042049 10590.73036587  4919.3533836  ... 10372.79600456
  4182.08392524  3961.63960982]
[8827.2130647  9751.00843879 4577.71906816 ... 3249.97689724 5372.48620994
 5645.14493884]
[6427.02752943 7298.6420734  6427.02752943 ... 6398.65221922 9046.0991074
 6398.65221922]
Test Error:
[17.247565076466156, 118.85820868958565, 18.222666862412304]
Training Error
[17.305973935341942, 17.58700788502257, 17.829561958820513]
Running GBT for...
n_estimators: 10
learning_rate:0.75
max_depth:3
[ 3964.6807144  10728.28905329  4590.68959428 ...  8832.80415266
  3571.72760249  3932.69892642]
[9397.76468895 8390.41711971 4585.62093219 ... 3621.39356369 5337.61161959
 5093.90274592]
[6333.61734054 7028.66414256 6333.61734054 ... 5179.24594418 9036.49412618
 6693.73559852]
Test Error:
[16.94772909318486, 16.840007290957555, 16.75480863546174]
Training Error
[16.749572929715903, 16.614644164934216, 18.182401588126726]
Running GBT fo

In [ ]:
## Took too long


parameters = {
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

clf = GridSearchCV(GradientBoostingRegressor(), parameters, cv=10, n_jobs=2)

clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.best_params_)
